<a href="https://colab.research.google.com/github/YinmiAlas/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-12 21:41:59--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.45.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.45.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  16.1MB/s    in 13s     

2020-05-12 21:42:14 (14.7 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [4]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [5]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [6]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####
'''i will be hones i did struggle it a lot was a lot code last night
but im trying to do my best '''

In [0]:
#reading both cvs testing and train
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')

In [8]:
#we need to concatening to have a better merge
order_products = pd.concat([order_products__prior, order_products__train], axis=0)
order_products.shape

(33819106, 4)

In [92]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
#this way is how you skip some cloums
myorder_products = order_products[['order_id', 'product_id', 'add_to_cart_order']]
myorder_products.head()

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5


In [15]:
#this is the data frame that i need
orders = pd.read_csv('orders.csv')
orders.sample(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1269713,1629450,76352,test,16,6,13,5.0
2888289,3214433,174280,prior,5,6,11,13.0
2844746,2200421,171653,prior,3,3,14,12.0
2758541,1407942,166265,prior,11,5,21,5.0
1675771,2182826,100598,prior,15,6,12,8.0
86332,2727668,5262,prior,13,2,15,30.0
2072664,2070002,124611,prior,20,4,22,14.0
1670830,2747984,100304,prior,14,3,12,7.0
585879,477081,35298,prior,14,5,18,2.0
534488,2517942,32236,prior,9,5,13,7.0


In [11]:
#this way is how you make another data frame skiping some columns
#just move the columns you need to another DataFrame 
myorders = orders[['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day']]
myorders.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day
0,2539329,1,1,2,8
1,2398795,1,2,3,7
2,473747,1,3,3,12
3,2254736,1,4,4,7
4,431534,1,5,4,15
5,3367565,1,6,2,7
6,550135,1,7,1,9
7,3108588,1,8,1,14
8,2295261,1,9,1,16
9,2550362,1,10,4,8


In [0]:
#my first merge i don't really get it whats ON mean beacause when i write it down 
#it was always an error so i decided skip it 
mymerge = pd.merge(myorder_products, myorders, how='inner')

In [13]:
mymerge.head(3)

,order_id,product_id,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day
0,2,33120,1,202279,3,5,9
1,2,28985,2,202279,3,5,9
2,2,9327,3,202279,3,5,9


In [49]:
#this is the second data frame 
products = pd.read_csv('products.csv')
products.sample(10)

,product_id,product_name,aisle_id,department_id
31471,31472,Chocolate Banana Granola Mix,57,14
47891,47892,Lightdays Liners,126,11
41852,41853,Red Hot Sweet Chili Hot Sauce,72,13
4965,4966,37% Less Sodium Soy Sauce,66,6
1741,1742,Onion Crispbread,100,21
4891,4892,Wedge Roquefort,2,16
31696,31697,Mild Hot Tortilla Chips,107,19
32106,32107,Organic Dark Chocolate Almonds,117,19
41540,41541,Traditional Seitan,14,20
49222,49223,Extreme Poison Ivy Scrub,83,4


In [50]:
myproducts = products[['product_id', 'product_name', 'aisle_id']]
myproducts.head(5)

,product_id,product_name,aisle_id
0,1,Chocolate Sandwich Cookies,61
1,2,All-Seasons Salt,104
2,3,Robust Golden Unsweetened Oolong Tea,94
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38
4,5,Green Chile Anytime Sauce,5


In [0]:
#this way is for select multiple rows 
#the only i need using .isin
top10 = myproducts[myproducts['product_name'].isin(['Banana',
                                        'Bag of Organic Bananas', 
                                        'Organic Strawberries', 
                                        'Organic Baby Spinach',
                                        'Organic Hass Avocado',
                                        'Organic Avocado',
                                        'Large Lemon',
                                        'Strawberries',
                                        'Limes',
                                        'Organic Whole Milk'])]

In [53]:
top10.head(15)

,product_id,product_name,aisle_id
13175,13176,Bag of Organic Bananas,24
16796,16797,Strawberries,24
21136,21137,Organic Strawberries,24
21902,21903,Organic Baby Spinach,123
24851,24852,Banana,24
26208,26209,Limes,24
27844,27845,Organic Whole Milk,84
47208,47209,Organic Hass Avocado,24
47625,47626,Large Lemon,24
47765,47766,Organic Avocado,24


In [54]:
top10.shape

(10, 3)

In [35]:
mymerge.head(2)

,order_id,product_id,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day
0,2,33120,1,202279,3,5,9
1,2,28985,2,202279,3,5,9


In [0]:
#selecting rows by product_id
top10_product_id = mymerge[mymerge['product_id'].isin([24852, 13176, 16797, 
                                                       21137, 21903, 26209, 
                                                       27845, 47626, 47209, 
                                                       47766])]

In [56]:
top10_product_id.shape

(2418314, 7)

In [41]:
top10_product_id.head(15)

,order_id,product_id,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day
12,3,21903,4,205970,16,5,17
30,5,13176,1,156122,42,6,16
77,10,24852,1,135442,4,6,8
83,10,21137,7,135442,4,6,8
126,14,27845,2,18194,49,3,15
180,20,24852,6,182912,1,6,17
190,22,24852,3,30822,25,1,8
234,26,24852,2,153404,2,0,16
238,26,21903,6,153404,2,0,16
241,27,13176,1,129389,16,6,20


In [0]:
finalmerge = pd.merge(top10_product_id, top10, how='inner')

In [58]:
finalmerge.shape

(2418314, 9)

In [66]:
finalmerge.head()

,order_id,product_id,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,product_name,aisle_id
0,3,21903,4,205970,16,5,17,Organic Baby Spinach,123
1,26,21903,6,153404,2,0,16,Organic Baby Spinach,123
2,31,21903,3,201744,7,6,15,Organic Baby Spinach,123
3,39,21903,4,184886,8,5,18,Organic Baby Spinach,123
4,56,21903,8,41591,8,0,12,Organic Baby Spinach,123


In [74]:
#getting the counts of the top 10 most 
#frequently ordered products.
finalmerge['product_name'].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [83]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [84]:
# table2 -> tidy
table2.columns.tolist()

['John Smith', 'Jane Doe', 'Mary Johnson']

In [85]:
table2.index.tolist()

['treatmenta', 'treatmentb']

In [86]:
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [88]:
#creating the table to tidy fortmat using melt
#i was a little confused beacause the last night 
#lesson was different but the same data 
# but i figured out and works out i just change the Value_vars= 
tidy_table2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])

tidy_table2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [89]:
tidy_table2 = tidy_table2.rename(columns={'index':'treat',
                                          'variable':'name',
                                          'value':'result'})
tidy_table2

,treat,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [94]:
# tidy -> table2
#now lets do the reverse bring everything back 
#agian like above
wide_table2 = tidy_table2.pivot_table(index='treat', columns='name', values='result')
wide_table2

name,Jane Doe,John Smith,Mary Johnson
treat,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [0]:
flights = sns.load_dataset('flights')

In [0]:
#Then create a pivot table showing the number of passengers by month and year.
# Use year for the index and month for the columns. 
#You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [102]:
# Flights Pivot Table
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [98]:
#creating a table using pivot_table
numbers_of_passenger = flights.pivot_table(index='year', columns='month', values='passengers')
numbers_of_passenger

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####


## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####